In [14]:
import json
import jieba
import parsel
import requests
from itertools import islice
from concurrent.futures import ThreadPoolExecutor

## 爬虫：《周易》384爻今释

In [3]:
url = 'http://www.360doc.com/content/18/0728/06/42860393_773759238.shtml'
resp = requests.get(url)
html = parsel.Selector(resp.text)
plist = html.xpath("//td[@id='artContent']/div[2]/p[position() >= 5]/text()").getall()
with open('data/iching_384_changes.txt', 'w', encoding='utf8') as wf:
    wf.writelines('\n'.join(plist))

In [2]:
severities = {
    '1.1': '学业烦恼、对未来规划的迷茫', '1.2': '事业和工作烦恼', '1.3': '家庭问题和矛盾', '1.4': '物质滥用',
    '1.5': '悲恸', '1.6': '失眠', '1.7': '压力', '1.8': '人际关系', '1.9': '情感关系问题', '1.10': '离婚',
    '1.11': '分手', '1.12': '自我探索', '1.13': '低自尊', '1.14': '青春期问题', '1.15': '强迫症', 
    '1.16': '其它', '1.17': '男同性恋、女同性恋、双性恋与跨性别', '1.18': '性问题', '1.19': '亲子关系'
}

# 学业 事业 家庭 情感 健康 决策

In [3]:
with open('../datasets/EFA_Dataset_v20200314_latest.txt', 'r', encoding='utf8') as rf:
    vocab = []
    while True:
        lines = list(islice(rf, 64))
        if not lines: break
        for line in lines:
            js = json.loads(line)
            title = js['title']
            label = severities[js['label']['s1']]
            print(label, title)
        break


低自尊 女 听过别人最多的议论就是干啥啥不行不长心眼没有脑子
其它 女 ─=≡Σ((( つ•̀Ω•́)つ 有喜欢看动漫的吗？
其它 女 有男的么，聊会天。。。。。。。。。。。。
其它 男 精力不集中，，，，，，，，，，，，，，，
其它 女 有人在吗，聊会。。。。。。。。。。。。。
其它 男 刚刚学会互联网，希望眼睛还好，，，，，，
其它 女 新冠肺炎病毒让人明白的人生真理。有图解释
其它 男 既学习互联网又遵守规则又不落伍，，，，，
其它 女 抑郁症单子大概解读，18岁，看不懂那个单子
失眠 女 我整宿整宿睡不着经常头疼不知道自己怎么了
其它 男 没有抑郁症的特征，就是想死不知道为什么。
其它 男 走路的时候，陌生人摸头发给我看，感到恐惧
情感关系问题 放不下女朋友的过去，又不想放手，我该怎么办
情感关系问题 女 你别装本来在乎了，我眼里的难过快溢出来了
情感关系问题 老婆与异性经常聊天是否正常？聊天语气很轻浮？
其它 女 想做的做不成，现在做的没意思饭啊做什么呢
其它 女 舍曲林突然加大了药量会死么，大概加了3，4倍
情感关系问题 女 如果给不了一个人幸福和快乐是不是该放手了
其它 女,现在还有没有没休息的老师？想简单做点咨询
其它 睡觉时身体动不了意识很清醒，这是什么问题？
青春期问题 女 我暗恋了一个男孩，可是不知道怎样接触他？
情感关系问题 女,爱上一个不该爱的人，想忘又忘不了，怎么办
压力 女 睡不着，老是想东想西，待在房间里有种恐惧
其它 男 我需要著名的心里故事,可以提供的话,,我付费
其它 女 闲着没事干，脑袋空空的，是病么，怎么改正
其它 女 想问问这是有抑郁症了吗？如果有了该怎么办
自我探索 女 从小到大脾气一直不好，如何让脾气好起来。
其它 男 价值观的形成，，，，，，，，，，，，，，
压力 女 如何治愈恐惧症和焦虑症？极度没有安全感！
其它 男 郁闷，，，，，，，，，，，，，，，，，，
其它 女 防备心很重，情感麻木的人。。。。。。。。
家庭问题和矛盾 女 如果改变不了老公，是不是只能改变我自己呀
其它 女 我能做的只剩下止损了！可怕。。。悔不当初
其它 女 说来话长可以帮助我吗我真的有事想问问你们
强迫症 女 女14强迫症和抑郁症有关系吗 强迫症怎么治疗
压力 男 我感觉自己抑郁了，最近很烦，看什么都生气.
人际关系 女 男朋友母亲老喜欢翻

In [11]:
url = 'https://www.guoyi360.com/64gua{:02d}/gs/'

urls = []
for i in range(1, 65):
    resp = requests.get(url.format(i))
    html = parsel.Selector(resp.text)
    hrefs = html.xpath("//div[@class='list-title']/h4/a/@href").getall()
    urls.append((i, ['https://www.guoyi360.com' + href for href in hrefs]))

In [17]:
urls = urls[2:]


In [51]:
with open('data/chings/IChing_newest.txt', 'r', encoding='utf8') as rf:
    hexagrams = [line.split('|')[1] for line in rf.readlines()]

In [53]:
def fetch_story(idx, urls):
    stories = []
    for url in urls:
        resp = requests.get(url)
        resp.encoding = 'utf8'
        html = parsel.Selector(resp.text)
        title = html.xpath("//div[@id='wenzhang-title']/h1/text()").get()
        plist = html.xpath("//div[@class='content']/p[position() > 2]/text()").getall()
        title = title.split('：')[0]
        fields = title.split('，')
        title = hexagrams[idx - 1] + ' ' + fields[1] + ' ' + fields[0]
        plist.insert(0, title)
        stories.append('\n'.join([p.strip() for p in plist[:-1]]))
    with open('data/384story/{}.txt'.format(idx), 'w',encoding='utf8') as wf:
        wf.writelines('\n\n'.join(stories))

with ThreadPoolExecutor(max_workers=16) as executor:
    results = executor.map(lambda p: fetch_story(*p), urls)
    stories = [result for result in results]

In [65]:
with open('data/384_changes_story.txt', 'a+', encoding='utf8') as wf:
    for i in range(3, 65):
        hexagram_name = hexagrams[i - 1]
        with open('datasets/384story/{}.txt'.format(i), 'r', encoding='utf8') as rf:
            lines = []
            for line in rf.readlines():
                if '】' in line:
                    line = line.split('】')[-1]
                lines.append(line)
            
            wf.writelines(''.join(lines) + '\n\n')

In [78]:
with open('data/384_changes_story.txt', 'r', encoding='utf8') as rf:
    lines = rf.readlines()
    num = len(lines)
    stories_tmp = []
    idx = 0
    while idx + 2 < num:
        if lines[idx].strip() == '' and lines[idx + 2].strip() == '':
            idx += 2
            stories_tmp.append('\n')
            continue
        stories_tmp.append(lines[idx])
        idx += 1


In [80]:
with open('data/tmp.txt', 'w',encoding='utf8') as wf:
    wf.writelines(''.join(stories_tmp).replace('\n\n', '\n'))